In [1]:
import pandas as pd
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
os.getcwd()

'D:\\workspace\\DACON_2019\\SUHYUN'

In [3]:
os.chdir('D:\\workspace\\DACON_2019')

In [4]:
test = pd.read_csv('.\\SOOMIN\\data\\Test_시간별평균대치.csv')
test

,Time,X26,X303,X241,X435,X402,X352,X305,X350,X326,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
0,2017-07-01 00:00:00,1.187315,0.385586,0.298691,0.276657,0.190045,0.160754,0.343880,0.249761,0.070329,...,0.288141,0.326095,0.356141,0.407326,0.159644,0.227077,0.168175,0.275,0.021,0.338138
1,2017-07-01 01:00:00,1.186123,0.478955,0.307983,0.333324,0.193712,0.167303,0.426744,0.293231,0.072641,...,0.259911,0.297695,0.296728,0.316149,0.159288,0.259118,0.184568,0.222,0.021,0.335306
2,2017-07-01 02:00:00,1.168301,0.467760,0.323560,0.331767,0.200656,0.165983,0.405878,0.301436,0.076477,...,0.237175,0.261525,0.288334,0.267332,0.157027,0.267059,0.177599,0.237,0.021,0.344922
3,2017-07-01 03:00:00,1.163425,0.443529,0.304879,0.322123,0.189067,0.155355,0.401216,0.288693,0.082617,...,0.203332,0.207759,0.253232,0.212221,0.154466,0.251081,0.156110,0.229,0.020,0.334552
4,2017-07-01 04:00:00,1.398466,0.390286,0.322174,0.320922,0.195500,0.154073,0.344145,0.295861,0.074067,...,0.189208,0.174952,0.266133,0.244444,0.152014,0.253930,0.143873,0.202,0.021,0.346241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-06-30 19:00:00,1.255000,0.175000,0.430000,0.429000,0.169000,0.739000,0.573000,0.341000,0.127000,...,0.312000,0.381000,0.479000,0.858000,11.546000,0.428000,0.134000,3.519,0.027,0.286000
8756,2018-06-30 20:00:00,1.596000,1.692000,0.332000,0.366000,0.159000,0.255000,0.501000,0.281000,0.126000,...,0.327000,0.402000,0.725000,0.816000,11.591000,0.413000,0.206000,3.331,0.027,0.303000
8757,2018-06-30 21:00:00,1.379000,0.883000,0.435000,0.497000,0.166000,0.233000,0.634000,0.401000,0.439000,...,0.253000,0.401000,0.426000,0.659000,10.766000,0.466000,0.324000,3.416,0.026,0.253000
8758,2018-06-30 22:00:00,1.000000,0.539000,0.268000,0.459000,0.463000,0.454000,0.676000,0.543000,0.737000,...,0.332000,0.401000,0.353000,0.462000,0.312000,0.473000,0.402000,4.263,0.027,0.253000


In [5]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))


In [6]:
test['Time'] = pd.to_datetime(test['Time']) 
test = test.set_index('Time')

In [7]:
signal_data = test.iloc[:, 0].to_frame()

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
signal_data = scaler.fit_transform(signal_data)

In [9]:
# 데이터 분리
train = signal_data[:5256]
val = signal_data[5256:7008]
test = signal_data[7008:]

In [10]:
look_back = 24

In [11]:
# 데이터셋 생성
x_train, y_train = create_dataset(train, look_back)
x_val, y_val = create_dataset(val, look_back)
x_test, y_test = create_dataset(test, look_back)

In [12]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [13]:
# 2. 모델 구성하기
model = Sequential()
for i in range(2):
    model.add(LSTM(32, batch_input_shape=(1, look_back, 1), stateful=True, return_sequences=True))
    model.add(Dropout(0.3))
model.add(LSTM(32, batch_input_shape=(1, look_back, 1), stateful=True))
model.add(Dropout(0.3))
model.add(Dense(1))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
early_callback = 

SyntaxError: invalid syntax (<ipython-input-14-c4b2f90621c3>, line 1)

In [15]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [16]:
# 4. 모델 학습시키기
custom_hist = CustomHistory()
custom_hist.init()

In [17]:
batch_size= 1

In [ ]:
for i in range(10):
    model.fit(x_train, y_train, epochs=1, batch_size=batch_size, shuffle=False, callbacks=[custom_hist], validation_data=(x_val, y_val))
    model.reset_states()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 5232 samples, validate on 1728 samples
Epoch 1/1
5232/5232 [==============================] - 153s 29ms/step - loss: 0.0020 - val_loss: 0.0014
Train on 5232 samples, validate on 1728 samples
Epoch 1/1
5232/5232 [==============================] - 182s 35ms/step - loss: 7.0152e-04 - val_loss: 2.4618e-04
Train on 5232 samples, validate on 1728 samples
Epoch 1/1
5232/5232 [==============================] - 197s 38ms/step - loss: 3.1818e-04 - val_loss: 3.0421e-04
Train on 5232 samples, validate on 1728 samples
Epoch 1/1
 862/5232 [===>..........................] - ETA: 2:36 - loss: 3.0433e-04- ETA: 2:38 - lo

In [ ]:
# making_shift_count = 25

# dataset = Series_data.to_frame()

# from sklearn.preprocessing import MinMaxScaler
# dataset_sc=MinMaxScaler().fit_transform(dataset)
# dataset_sc_df = pd.DataFrame(dataset_sc, columns=dataset.columns, index = dataset.index)

# for s in range(1, making_shift_count):
#     dataset_sc_df['shift_{}'.format(s)] = dataset_sc_df[dataset.columns].shift(s)

# X_dataset = dataset_sc_df.dropna().drop(dataset.columns, axis=1)
# y_dataset = dataset_sc_df.dropna()[dataset.columns]

# X_dataset = X_dataset.values
# y_dataset = y_dataset.values

# X_dataset_t = X_dataset.reshape(X_dataset.shape[0], making_shift_count-1, 1)

In [ ]:
del model

In [ ]:
from keras.layers import LSTM 
from keras.models import Sequential 
from keras.layers import Dense 
import keras.backend as K 
from keras.callbacks import EarlyStopping

In [ ]:
K.clear_session()
    
model = Sequential() # Sequeatial Model 
model.add(LSTM(20, input_shape=(X_dataset_t.shape[1], 1))) # (timestep, feature) 
model.add(Dense(1)) # output = 1 
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy']) 
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=1, verbose=1)

history = model.fit(X_dataset_t, y_dataset, epochs=100, batch_size=30, verbose=1, callbacks=[early_stop])

In [ ]:
y_vloss=history.history['val_loss'] # 오차 표시를 위해 저장
y_acc=history.history['acc'] # 정확도 표시를 위해 저장

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x_len = np.arange(len(y_acc))
plt.plot(x_len, y_vloss, "o", c="red", markersize=3) # 빨간색은 오차
plt.plot(x_len, y_acc, "o", c="blue", markersize=3) # 파란색은 정확도
plt.show()